# UEP-0239: Introduction to Geospatial Python
---

## Working with GeoPandas

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
stops = pd.read_csv('data/mbta-rapid-transit-stations.csv')

In [ ]:
stops

In [ ]:
stops = gpd.GeoDataFrame(stops,
                         geometry=gpd.points_from_xy(stops.stop_lon,
                                                     stops.stop_lat))

In [ ]:
stops.head()

In [ ]:
type(stops)

In [ ]:
stops.crs

In [ ]:
stops.set_crs(epsg=4326, inplace=True)

In [ ]:
stops.crs

In [ ]:
stops.to_crs(epsg=26986, inplace=True)

In [ ]:
stops.head()

---

## Creating Static Maps

In [ ]:
import matplotlib.pyplot as plt
import contextily as cx
import seaborn as sns

In [ ]:
stops.plot()
plt.show()

In [ ]:
ax = stops.plot(figsize=(10, 10), color='black', markersize=50)
cx.add_basemap(ax, crs=stops.crs)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.kdeplot(x=stops.geometry.x,
            y=stops.geometry.y,
            fill=True,
            alpha=0.75,
            cmap='magma',
            ax=ax)
cx.add_basemap(ax, crs=stops.crs)
plt.show()

In [ ]:
ax = stops[stops.accessible == 'yes'].plot(figsize=(10, 10),
                                           color='blue',
                                           label='accessilbe',
                                           marker='o',
                                           markersize=50)
stops[stops.accessible == 'no'].plot(color='red',
                                     label='not accessible',
                                     marker='x',
                                     markersize=100,
                                     ax=ax)
plt.legend()
cx.add_basemap(ax, crs=stops.crs)
plt.show()

---

## Creating Interactive Maps

In [ ]:
stops.explore(column='accessible',
              tooltip='stop_name',
              tooltip_kwds=dict(labels=False),
              marker_kwds=dict(radius=5, fill=True),
              popup=True,
              legend=True,
              cmap='bwr_r',
              tiles='CartoDB positron')

---

## Exploring Shapely Objects

In [ ]:
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.ops import transform
import pyproj
import folium

In [ ]:
stops[stops.stop_name == 'Davis']

In [ ]:
stops[stops.stop_name == 'Davis'].geometry

In [ ]:
davis = stops[stops.stop_name == 'Davis'].geometry.values[0]

In [ ]:
davis

In [ ]:
type(davis)

In [ ]:
plt.scatter(davis.x, davis.y)
plt.show()

In [ ]:
porter = stops[stops.stop_name == 'Porter'].geometry.values[0]
harvard = stops[stops.stop_name == 'Harvard'].geometry.values[0]

In [ ]:
ax = stops.plot(figsize=(10, 10), color='black', markersize=25)
ax.scatter(davis.x, davis.y, color='blue', marker='*', s=200)
ax.scatter(porter.x, porter.y, color='green', marker='X', s=100)
ax.scatter(harvard.x, harvard.y, color='red', marker='s', s=50)
cx.add_basemap(ax, crs=stops.crs, source=cx.providers.CartoDB.Positron)
plt.show()

In [ ]:
route = LineString([davis, porter, harvard])

In [ ]:
route

In [ ]:
type(route)

In [ ]:
route.length

In [ ]:
davis.distance(harvard)

In [ ]:
x, y = route.xy
plt.plot(x, y)
plt.show()

In [ ]:
x, y = route.xy
ax = stops.plot(figsize=(10, 10),color='black', markersize=50, zorder=2)
plt.plot(x, y, color='red', linewidth=3, zorder=1)
cx.add_basemap(ax, crs=stops.crs, source=cx.providers.CartoDB.Positron)
plt.show()

In [ ]:
transformer = pyproj.Transformer.from_crs(stops.crs, 'EPSG:4326').transform
m = folium.Map(location=transform(transformer, route).centroid.coords[0],
               tiles='CartoDB positron',
               zoom_start=14)
folium.PolyLine(locations=transform(transformer, route).coords,
                color='red').add_to(m)
folium.Marker(location=transform(transformer, davis).coords[0],
              tooltip='Davis').add_to(m)
folium.Marker(location=transform(transformer, porter).coords[0],
              tooltip='Porter').add_to(m)
folium.Marker(location=transform(transformer, harvard).coords[0],
              tooltip='Harvard').add_to(m)
m

---

## Geocoding with GeoPy

In [ ]:
import random
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
stops2 = pd.read_csv('data/mbta-rapid-transit-stations.csv')

In [ ]:
stops2 = stops2[~stops2.stop_address.str.contains('and|at', na=True)].reset_index(drop=True)

In [ ]:
stops2.head()

In [ ]:
stops2.shape

In [ ]:
stops2 = stops2.head(20).copy()

In [ ]:
token = 'uep239-python-spatial-{:04}'.format(random.randint(0,9999))
print(token)

In [ ]:
geolocator = Nominatim(user_agent=token)

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
stops2['location'] = stops2.stop_address.apply(geocode)

In [ ]:
stops2.head()

In [ ]:
stops2['geoloc_y'] = stops2.location.apply(lambda loc: loc.point[0] if loc else None)
stops2['geoloc_x'] = stops2.location.apply(lambda loc: loc.point[1] if loc else None)

In [ ]:
stops2.drop(columns='location', inplace=True)

In [ ]:
stops2

---

## Exercise

In [ ]:
entries = (pd.read_csv('data/mbta-gated-entries-2021.csv')
             .groupby('stop_id')
             .gated_entries.sum()
             .to_frame('total_entries')
             .reset_index())

In [ ]:
entries